<a href="https://colab.research.google.com/github/daljee-T/Mini-Projects-11--26-ML-/blob/main/ImageDownloader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Step 1: Install dependencies
!pip install selenium webdriver-manager pillow
!apt-get update # for Chromium driver
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

  Using cached selenium-4.33.0-py3-none-any.whl.metadata (7.5 kB)
  Using cached webdriver_manager-4.0.2-py2.py3-none-any.whl.metadata (12 kB)
  Using cached trio-0.30.0-py3-none-any.whl.metadata (8.5 kB)
  Using cached trio_websocket-0.12.2-py3-none-any.whl.metadata (5.1 kB)
  Using cached typing_extensions-4.13.2-py3-none-any.whl.metadata (3.0 kB)
  Using cached python_dotenv-1.1.0-py3-none-any.whl.metadata (24 kB)
  Using cached outcome-1.3.0.post0-py2.py3-none-any.whl.metadata (2.6 kB)
Using cached selenium-4.33.0-py3-none-any.whl (9.4 MB)
Using cached webdriver_manager-4.0.2-py2.py3-none-any.whl (27 kB)
Using cached trio-0.30.0-py3-none-any.whl (499 kB)
Using cached trio_websocket-0.12.2-py3-none-any.whl (21 kB)
Using cached typing_extensions-4.13.2-py3-none-any.whl (45 kB)
Using cached python_dotenv-1.1.0-py3-none-any.whl (20 kB)
Using cached outcome-1.3.0.post0-py2.py3-none-any.whl (10 kB)
  Attempting uninstall: typing_extensions
    Found existing installation: typing_extensio

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:6 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
E: dpkg was interrupted, you must manually run 'dpkg --configure -a' t

In [ ]:
# Step 2: Import libraries
import os
import time
import requests
from PIL import Image
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager

# Step 3: Setup Chrome options for headless browsing
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--no-sandbox")

# Step 4: Function to download images
def download_images(query, num_images, download_path="downloads"):
    driver = webdriver.Chrome(options=chrome_options)  # ✅ Fixed line
    search_url = "https://www.google.com/search?q={}&tbm=isch".format(query)
    driver.get(search_url)

    os.makedirs(download_path, exist_ok=True)

    # Scroll to load more images
    for _ in range(5):
        driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.END)
        time.sleep(1)

    # Get image elements
    image_elements = driver.find_elements(By.CSS_SELECTOR, "img.rg_i")

    count = 0
    for i, img in enumerate(image_elements):
        if count >= num_images:
            break
        try:
            img.click()
            time.sleep(1)
            actual_images = driver.find_elements(By.CSS_SELECTOR, 'img.n3VNCb')
            for actual_image in actual_images:
                src = actual_image.get_attribute('src')
                if src and "http" in src:
                    try:
                        image_data = requests.get(src).content
                        file_path = os.path.join(download_path, f"{query}_{count+1}.jpg")
                        with open(file_path, 'wb') as f:
                            f.write(image_data)
                        count += 1
                        print(f"Downloaded {count}: {file_path}")
                        break
                    except:
                        continue
        except:
            continue
    driver.quit()
